### Next Word Predictor Using LSTM

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader('/content/englishliterature.pdf')

page = reader.pages[0]
faqs = ""
for page in reader.pages:
  faqs +=  page.extract_text()

13


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts([faqs])

In [ ]:
len(tokenizer.word_index)

2751

In [ ]:
input_sequence = []
for sentence in faqs.split("\n"):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1, len(tokenized_sentence)):
    input_sequence.append(tokenized_sentence[:i+1])

In [ ]:
max_len = max([len(x) for x in input_sequence])
max_len

31

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequence, maxlen = max_len, padding="pre")

In [ ]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,    8,    9],
       [   0,    0,    0, ...,    0,    8,    9],
       [   0,    0,    0, ...,    0,   38,   81],
       ...,
       [   0,    0,    0, ...,  835,   12, 2751],
       [   0,    0,    0, ...,   12, 2751,  133],
       [   0,    0,    0, ..., 2751,  133,  837]], dtype=int32)

In [ ]:
X = padded_input_sequences[:,:-1]

In [ ]:
y = padded_input_sequences[:,-1]

In [ ]:
X.shape, y.shape

((7592, 30), (7592,))

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=2752)

In [ ]:
y.shape

(7592, 2752)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
model = Sequential()
model.add(Embedding(2752, 100, input_length=30))
model.add(LSTM(150))
model.add(Dense(2752, activation= "softmax"))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 30, 100)           275200    
                                                                 
 lstm_4 (LSTM)               (None, 150)               150600    
                                                                 
 dense_2 (Dense)             (None, 2752)              415552    
                                                                 
Total params: 841352 (3.21 MB)
Trainable params: 841352 (3.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X,y,epochs=100)

Epoch 1/100
238/238 [==============================] - 20s 73ms/step - loss: 6.9924 - accuracy: 0.0703
Epoch 2/100
238/238 [==============================] - 18s 76ms/step - loss: 6.4162 - accuracy: 0.0876
Epoch 3/100
238/238 [==============================] - 18s 75ms/step - loss: 6.1865 - accuracy: 0.0969
Epoch 4/100
238/238 [==============================] - 17s 73ms/step - loss: 5.9259 - accuracy: 0.1089
Epoch 5/100
238/238 [==============================] - 19s 78ms/step - loss: 5.6588 - accuracy: 0.1214
Epoch 6/100
238/238 [==============================] - 17s 73ms/step - loss: 5.3920 - accuracy: 0.1350
Epoch 7/100
238/238 [==============================] - 17s 72ms/step - loss: 5.1283 - accuracy: 0.1463
Epoch 8/100
238/238 [==============================] - 18s 77ms/step - loss: 4.8727 - accuracy: 0.1585
Epoch 9/100
238/238 [==============================] - 17s 73ms/step - loss: 4.6249 - accuracy: 0.1698
Epoch 10/100
238/238 [==============================] - 18s 77ms/step - l

In [ ]:
import time
import numpy as np
text = "An employee’s services may be terminated"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=30, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 [==============================] - 0s 440ms/step
An employee’s services may be terminated works
1/1 [==============================] - 0s 23ms/step
An employee’s services may be terminated works of
1/1 [==============================] - 0s 33ms/step
An employee’s services may be terminated works of the
1/1 [==============================] - 0s 24ms/step
An employee’s services may be terminated works of the 20th
1/1 [==============================] - 0s 26ms/step
An employee’s services may be terminated works of the 20th century
1/1 [==============================] - 0s 23ms/step
An employee’s services may be terminated works of the 20th century saw
1/1 [==============================] - 0s 26ms/step
An employee’s services may be terminated works of the 20th century saw several
1/1 [==============================] - 0s 25ms/step
An employee’s services may be terminated works of the 20th century saw several major
1/1 [==============================] - 0s 45ms/step
An employee’s servi